In [1]:
# Import libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
# Connect to local cluster at port 8080
spark = SparkSession.builder \
    .master("spark://de-zoomcamp.australia-southeast1-b.c.taxi-rides-ny-412407.internal:7077") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/02 01:45:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_green = spark.read.parquet('../data/pq/green/*/*')

In [7]:
df_green_rename = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime') \
    .withColumnRenamed('PULocationID', 'pickup_id') \
    .withColumnRenamed('DOLocationID', 'dropoff_id') \
    .withColumnRenamed('RatecodeID', 'ratecode_id') \
    .withColumnRenamed('VendorID', 'vendor_id')

In [8]:
df_yellow = spark.read.parquet('../data/pq/yellow/*/*')

In [9]:
df_yellow_rename = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime') \
    .withColumnRenamed('PULocationID', 'pickup_id') \
    .withColumnRenamed('DOLocationID', 'dropoff_id') \
    .withColumnRenamed('RatecodeID', 'ratecode_id') \
    .withColumnRenamed('VendorID', 'vendor_id')

In [10]:
common_cols = list()

yellow_cols = set(df_yellow_rename.columns)

for col in df_green_rename.columns:
    if col in yellow_cols:
        common_cols.append(col)

In [11]:
from pyspark.sql import functions as F

In [12]:
df_green_sel = df_green_rename \
    .select(common_cols) \
    .withColumn('service_type', F.lit('green'))

In [13]:
df_yellow_sel = df_yellow_rename \
    .select(common_cols) \
    .withColumn('service_type', F.lit('yellow'))

In [14]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [15]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [17]:
df_trips_data.createOrReplaceTempView('trips_data')

In [18]:
df_result = spark.sql("""

SELECT 
    -- Revenue grouping 
    pickup_id AS revenue_zone,
    DATE_TRUNC('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,

    -- Additional calculations
    AVG(passenger_count) AS avg_monthly_passenger_count,
    AVG(trip_distance) AS avg_monthly_trip_distance

FROM 
    trips_data
GROUP BY
    revenue_zone,
    revenue_month,
    service_type

""")

In [19]:
df_result.count()

9895

In [20]:
df_result.coalesce(1).write.parquet('../data/report/revenue/', mode='overwrite')